In [1]:
import numpy as np
import math
import itertools
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import qiskit
import qiskit.quantum_info as qi 
# import warnings
# warnings.filterwarnings('ignore')
import time
import networkx as nx
from scipy.linalg import expm
from scipy.optimize import minimize, Bounds
from qiskit import QuantumCircuit, Aer, execute


In [2]:
from itertools import product
from functools import reduce

PAULIS = {'I': np.eye(2),
          'X': np.array([[0, 1], [1, 0]]),
          'Y': np.array([[0, -1j], [1j, 0]]),
          'Z': np.diag([1, -1])}

def get_pauli_decomp(in_matrix, display_every=5000, PAULIS=PAULIS):
    """Return dictionary with pauli strings and according weight.
    in_matrix - input matrix (should be of size(2^m,2^m), where m is int.
    display_time - if given int number print time spend for computation if False do not print.
    PAULIS - dictionary with Pauli basis.

    """
    m = int(np.log2(in_matrix.shape[0]))

    pauli_weights = {}

    k = 1
    K = len(list(product(PAULIS.keys(), repeat=m)))

    start_ = time.time()
    for u in product(PAULIS.keys(), repeat=m):
        pauli_str_name = ''.join(u)
        pauli_str_matrix = reduce(np.kron, [PAULIS[s] for s in u])
        inner_product = np.trace(in_matrix @ pauli_str_matrix) / 2**m
        if not np.isclose(inner_product, 0):
            pauli_weights[pauli_str_name] = inner_product
        if display_every and k%display_every == 0:
            print('\t {} qubits || {}/{} || {:.3f} s passed'.format(m,k,K, time.time()-start_))
        k+=1

    return pauli_weights

# produce all binary strings of length n with k 1s. If k is None then all possible binary strings of length n produced
def get_binary_strings(n, k=None) -> list[list]:
    '''
    produce all binary strings of length n with k 1s
    returns list with binary lists 
    '''
    final = []
    def kbits(r):
        result = []
        for bits in itertools.combinations(range(n), r):
            s = [0] * n
            for bit in bits:
                s[bit] = 1
            result.append(s)   
        return result

    if k != None:
        return kbits(k)
    
    for i in range(n + 1):
        final = final + kbits(i)
        
    return final

def count_solutions_XI(nqubits: int, Operator: np.array, tol=1e-10, if_print = False ) -> tuple:
    
    x_strings = get_binary_strings(nqubits)
    z_zeros = np.zeros(nqubits)

    ans = []
    max_locality  = 0
    avg_locality = 0.0

    for i in x_strings:
        x_mat = get_circuit_operators(i,z_zeros)
        coef = 1/(1<<nqubits) * np.trace(x_mat@Operator)
        if np.abs(coef)>tol:
            count_x = np.sum(i)
            ans.append((str(i),coef))
            if count_x > max_locality:
                max_locality = count_x
            avg_locality += count_x
    len_ans = len(ans)
    if len_ans == 0:
        len_ans = 1
        print("len_ans is zero")
    
    if if_print:
        print("Non-zero Pauli strings:", len_ans)
        print("Max locality:", max_locality)
        print("Avg locality:", avg_locality/len_ans)
    return ans, max_locality, avg_locality/len_ans, len_ans



def get_circuit_operators( lst_x=None, lst_z=None):
    '''
    note the order! (x,z)
    '''
    # returns for example X @ I @ I @ X @ I where @ is tensor product
    return  qi.Pauli((lst_z,lst_x)).to_matrix()


def generate_QAOA_operator(H1,H2, number_of_layers = 1 , beta_angle = [], gamma_angle = []):
    """_summary_

    Args:
        H (_type_): _description_
        number_of_layers (int, optional): _description_. Defaults to 1.
        beta_angle (list, optional): _description_. Defaults to [].
        gamma_angle (list, optional): _description_. Defaults to [].

    Returns:
        _type_: _description_
    """    '''
    '''
    # define angles for mixers 
    if len(beta_angle)==0:
        beta_angle = np.random.rand(number_of_layers) * np.pi/3


    if len(gamma_angle)==0:
        gamma_angle = np.random.rand(number_of_layers) * np.pi/6

    ans = H1

    nqubits = int(np.log2(len(H1)))

   
    mixer = 0
    x_string = np.zeros(nqubits)

    for i in range(0,nqubits):
        x_string[i] = 1
        mixer = mixer + get_circuit_operators(x_string, np.zeros(nqubits))
        x_string[i] = 0
    for p in range(number_of_layers):
        # Making unitary for Hamiltonian exp
        exp_H = expm(1j*H2*beta_angle[p])
        # Making mixer X
        exp_X = expm(1j*mixer*gamma_angle[p])

        ans = exp_H @ exp_X @ ans @ exp_X.T.conjugate() @ exp_H.T.conjugate()

    return ans
   
def make_H_maxCUT(n, adj_mat):
    H = np.zeros((2**n,2**n),dtype=complex)
    zeros = np.zeros(n)

    for i in range(n):
        for j in range(i+1,n):
            if adj_mat[i][j]!=0:
                buf = np.zeros(n)
                buf[i] = buf[j] = 1
                H += adj_mat[i][j]*get_circuit_operators(zeros, buf)
    return H

In [3]:
H = make_H_maxCUT(2, [[0,1],[1,0]])
H

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]])

In [4]:
get_pauli_decomp(generate_QAOA_operator(H,H))

{'IX': (0.21888757104515466+0j),
 'XI': (0.21888757104515466+0j),
 'YY': (0.11342016220018172-1.734723475976807e-18j),
 'YZ': (0.22944337046125035+0j),
 'ZY': (0.22944337046125035+0j),
 'ZZ': (0.8865798377998182+0j)}

In [5]:
N = 4
# H = np.diag(np.random.rand(2**N))
# generate_QAOA_operator(H)

In [6]:
def mixed_Hamiltonian(N,*localities):
    rng = np.random.default_rng()
    H = 0
    for l in localities:
        Zs = get_binary_strings(N,l)
        pickedZ = rng.choice(Zs, min(5,len(Zs)), replace=False)
        for i in range(0,len(pickedZ)):
            H = H + get_circuit_operators(np.zeros(N),pickedZ[i])
    return H

In [7]:
rng = np.random.default_rng()
Zs = get_binary_strings(N,2)
# pickedZ = np.random.choice(Zs,size=number_of_terms,replace=False)
pickedZ = rng.choice(Zs, 4, replace=False)

# Making Hamiltonian
H1 = 0
for i in range(0,len(pickedZ)):
    H1 = H1 + get_circuit_operators(np.zeros(N),pickedZ[i])

buf = np.copy(H1)

In [8]:
Zs = get_binary_strings(N,3)
# pickedZ = np.random.choice(Zs,size=number_of_terms,replace=False)
pickedZ = rng.choice(Zs, 4, replace=False)

# Making Hamiltonian
H2 = buf
for i in range(0,len(pickedZ)):
    H2 = H2 + get_circuit_operators(np.zeros(N),pickedZ[i])

In [9]:
count_solutions_XI(N,generate_QAOA_operator(H2,H2,2))

([('[1, 0, 0, 0]', (0.3331259153330598-2.0816681711721685e-17j)),
  ('[0, 1, 0, 0]', (-0.10394951930596796-5.551115123125783e-17j)),
  ('[0, 0, 1, 0]', (0.3331259153330598+7.806255641895632e-18j)),
  ('[0, 0, 0, 1]', (0.09993830343129137-9.627715291671279e-17j)),
  ('[1, 1, 0, 0]', (-0.14318969785955948+2.7755575615628914e-17j)),
  ('[1, 0, 1, 0]', (0.39143089633967193-1.0408340855860843e-17j)),
  ('[1, 0, 0, 1]', (-0.363406205277138+4.163336342344337e-17j)),
  ('[0, 1, 1, 0]', (-0.14318969785955946+0j)),
  ('[0, 1, 0, 1]', (0.29685458165692574+3.469446951953614e-17j)),
  ('[0, 0, 1, 1]', (-0.3634062052771381+5.4643789493269423e-17j)),
  ('[1, 1, 1, 0]', (0.146899361168565+6.938893903907228e-18j)),
  ('[1, 1, 0, 1]', (-0.08575613539149914-2.0816681711721685e-17j)),
  ('[1, 0, 1, 1]', (0.3081294971022951+8.673617379884035e-18j)),
  ('[0, 1, 1, 1]', (-0.0857561353914991-4.85722573273506e-17j)),
  ('[1, 1, 1, 1]', (-0.027919089054135884-6.938893903907228e-18j))],
 4,
 2.1333333333333333,


In [10]:
get_pauli_decomp(generate_QAOA_operator(H1,H1))

{'IIIX': (-0.246030668994551+6.938893903907228e-18j),
 'IIXI': (0.2586307651120624-1.5612511283791264e-17j),
 'IXII': (-0.24603066899455106+1.6929650812821594e-17j),
 'IXIX': (0.07395913462704876-8.673617379884035e-19j),
 'IYIY': (0.021624216912229124+2.6020852139652106e-18j),
 'IYIZ': (-0.1609847333193132-1.4016526090003033e-18j),
 'IZIY': (-0.16098473331931318+6.938893903907228e-18j),
 'IZIZ': (0.9044166484607221-1.3877787807814457e-17j),
 'XIII': (0.17553332274588085+1.214306433183765e-17j),
 'XIIX': (-0.035177979237440316+2.6020852139652106e-18j),
 'XIYZ': (-0.03999135368821688+2.168404344971009e-18j),
 'XIZY': (0.019021517988485462+0j),
 'XIZZ': (-0.08309744236618152+1.9081958235744878e-17j),
 'XXII': (-0.03517797923744032+2.168404344971009e-18j),
 'XXZZ': (-0.03517797923744032+2.168404344971009e-18j),
 'XYIZ': (0.01902151798848547+3.0357660829594124e-18j),
 'XYZI': (0.01902151798848547+3.903127820947816e-18j),
 'XZIY': (0.01902151798848546+0j),
 'XZIZ': (-0.08309744236618152+8.67

In [11]:
mixed_Hamiltonian(2,1,2)

array([[ 3.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]])

In [12]:
N = 5
H = mixed_Hamiltonian(N,1,2,3)
ans , *_= count_solutions_XI(N,generate_QAOA_operator(H,H,1))
# print( np.ones(N) in ans)
ans

[('[1, 0, 0, 0, 0]', (0.003439629714838896+1.734723475976807e-18j)),
 ('[0, 1, 0, 0, 0]', (-0.12524331597427849-2.7755575615628914e-17j)),
 ('[0, 0, 1, 0, 0]', (-0.20219799838466307+1.734723475976807e-17j)),
 ('[0, 0, 0, 1, 0]', (-0.5089878934850092-1.6263032587282567e-17j)),
 ('[0, 0, 0, 0, 1]', (0.00539448844286039-3.0357660829594124e-17j)),
 ('[1, 1, 0, 0, 0]', (0.15806484886612718+0j)),
 ('[1, 0, 1, 0, 0]', (-0.12989153064313427+0j)),
 ('[1, 0, 0, 1, 0]', (0.03149847146473386-1.3877787807814457e-17j)),
 ('[1, 0, 0, 0, 1]', (-0.0823194535354022-1.9081958235744878e-17j)),
 ('[0, 1, 1, 0, 0]', (0.08366524898219707-1.734723475976807e-17j)),
 ('[0, 1, 0, 1, 0]', (0.07076313063032562+1.734723475976807e-17j)),
 ('[0, 1, 0, 0, 1]', (-0.021401234398460638-1.5612511283791264e-17j)),
 ('[0, 0, 1, 1, 0]', (0.09805044627973107+3.469446951953614e-18j)),
 ('[0, 0, 1, 0, 1]', (-0.021401234398460624-5.204170427930421e-18j)),
 ('[0, 0, 0, 1, 1]', (-0.05898044685919286-1.4493732293765994e-17j)),
 ('[

1   -- ,
2   --,
3   --,

1+2  2,
1+3  2,
2+3  2,

In [48]:
N = 5
H = mixed_Hamiltonian(N,2)

In [49]:
beta_angle = np.random.rand(10) * np.pi/3

gamma_angle = np.random.rand(10) * np.pi/6

ans = np.copy(H)

nqubits = int(np.log2(len(H)))

mixer = 0
x_string = np.zeros(nqubits)

for i in range(0,nqubits):
    x_string[i] = 1
    mixer = mixer + get_circuit_operators(x_string, np.zeros(nqubits))
    x_string[i] = 0
p=0

In [50]:
# Making unitary for Hamiltonian exp
exp_H = expm(1j*H*beta_angle[p])
# Making mixer X
exp_X = expm(1j*mixer*gamma_angle[p])

ans = exp_H @ exp_X @ ans @ exp_X.T.conjugate() @ exp_H.T.conjugate()

p+=1


In [51]:
xs = get_circuit_operators(np.ones(nqubits), np.zeros(nqubits))

In [52]:
exp_H @ xs == xs @ exp_H

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [53]:
beta = np.random.rand(1) * np.pi/3
beta_p = np.pi/2 - beta 
beta_p

array([1.33841923])

In [56]:
np.cos(beta), np.sin(beta_p), np.sin(beta), np.cos(beta_p)

(array([0.97312172]),
 array([0.97312172]),
 array([0.23029137]),
 array([0.23029137]))

In [57]:
exp_Hx0 = expm(mixer*beta)
exp_Hxp = expm(mixer*beta_p)

In [58]:
exp_Hx0@xs 

array([[7.08719603e-04+0.j, 3.10456874e-03+0.j, 3.10456874e-03+0.j, ...,
        2.60964711e-01+0.j, 2.60964711e-01+0.j, 1.14316421e+00+0.j],
       [3.10456874e-03+0.j, 7.08719603e-04+0.j, 1.35996620e-02+0.j, ...,
        5.95737514e-02+0.j, 1.14316421e+00+0.j, 2.60964711e-01+0.j],
       [3.10456874e-03+0.j, 1.35996620e-02+0.j, 7.08719603e-04+0.j, ...,
        1.14316421e+00+0.j, 5.95737514e-02+0.j, 2.60964711e-01+0.j],
       ...,
       [2.60964711e-01+0.j, 5.95737514e-02+0.j, 1.14316421e+00+0.j, ...,
        7.08719603e-04+0.j, 1.35996620e-02+0.j, 3.10456874e-03+0.j],
       [2.60964711e-01+0.j, 1.14316421e+00+0.j, 5.95737514e-02+0.j, ...,
        1.35996620e-02+0.j, 7.08719603e-04+0.j, 3.10456874e-03+0.j],
       [1.14316421e+00+0.j, 2.60964711e-01+0.j, 2.60964711e-01+0.j, ...,
        3.10456874e-03+0.j, 3.10456874e-03+0.j, 7.08719603e-04+0.j]])

In [60]:
exp_Hxp/10

array([[3.51263144+0.j, 3.06052782+0.j, 3.06052782+0.j, ...,
        2.0243592 +0.j, 2.0243592 +0.j, 1.76380807+0.j],
       [3.06052782+0.j, 3.51263144+0.j, 2.66661354+0.j, ...,
        2.32339916+0.j, 1.76380807+0.j, 2.0243592 +0.j],
       [3.06052782+0.j, 2.66661354+0.j, 3.51263144+0.j, ...,
        1.76380807+0.j, 2.32339916+0.j, 2.0243592 +0.j],
       ...,
       [2.0243592 +0.j, 2.32339916+0.j, 1.76380807+0.j, ...,
        3.51263144+0.j, 2.66661354+0.j, 3.06052782+0.j],
       [2.0243592 +0.j, 1.76380807+0.j, 2.32339916+0.j, ...,
        2.66661354+0.j, 3.51263144+0.j, 3.06052782+0.j],
       [1.76380807+0.j, 2.0243592 +0.j, 2.0243592 +0.j, ...,
        3.06052782+0.j, 3.06052782+0.j, 3.51263144+0.j]])

In [40]:
np.trace(exp_Hx0@xs)/2**N

(1.502285023644781+0j)

In [38]:
exp_Hxp

array([[2.54671366+0.j, 1.42244636+0.j, 1.42244636+0.j, ...,
        0.24785817+0.j, 0.24785817+0.j, 0.13843918+0.j],
       [1.42244636+0.j, 2.54671366+0.j, 0.79449593+0.j, ...,
        0.44375929+0.j, 0.13843918+0.j, 0.24785817+0.j],
       [1.42244636+0.j, 0.79449593+0.j, 2.54671366+0.j, ...,
        0.13843918+0.j, 0.44375929+0.j, 0.24785817+0.j],
       ...,
       [0.24785817+0.j, 0.44375929+0.j, 0.13843918+0.j, ...,
        2.54671366+0.j, 0.79449593+0.j, 1.42244636+0.j],
       [0.24785817+0.j, 0.13843918+0.j, 0.44375929+0.j, ...,
        0.79449593+0.j, 2.54671366+0.j, 1.42244636+0.j],
       [0.13843918+0.j, 0.24785817+0.j, 0.24785817+0.j, ...,
        1.42244636+0.j, 1.42244636+0.j, 2.54671366+0.j]])

In [ ]:
def plus_state(n):
    return 1/np.sqrt(2**n) * np.ones(2**n)

In [ ]:
def output_state(theta, H, layer):
    n = int(np.log2(len(H)))
    circ = generate_QAOA_operator(H, number_of_layers = layer , beta_angle = theta[:len(theta)//2], gamma_angle = theta[len(theta)//2:])
    return np.dot(plus_state(n).T,circ)


def energy(theta, H, layer):
    state = output_state(theta, H, layer)
    n = int(np.log2(len(H)))
    return np.real(np.dot(state.conjugate().T, np.dot(H, state)))


def get_expectation(H, layer):

    def execute_circ(theta):
        return energy(theta, H, layer)

    return execute_circ


In [ ]:
def theta_for_ground_state(H,layer=1, repeat=10, maxiter=100):
    bds = Bounds(0, 2 * np.pi)
    expectation = get_expectation(H,layer)
    for i in range(repeat):
        initial_params = np.random.uniform(0, np.pi/4, layer*2)  # initial guess of parameters
        res = minimize(expectation,
                       initial_params, method='L-BFGS-B', jac='3-point', bounds=bds, options={'maxiter': maxiter})
        if i == 0:
            E_min = res.fun
            params = res.x
        if res.fun < E_min:
            E_min = res.fun
            params = res.x
        # print(E_min)
    return params, E_min